In [ ]:
import fiftyone.utils.openimages as fouo

classes = fouo.get_classes()
print(classes)

segmentation_classes = fouo.get_segmentation_classes()
print(segmentation_classes)


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.openimages as fouo

classes = fouo.get_classes()

dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    classes=["Car", "Box", "Person"] ,
    max_samples=200,
    shuffle=True,
)
session = fo.launch_app(dataset)

In [ ]:
export_dir = "openimg"  

dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.FiftyOneDataset, 
)

In [16]:
import json

# Load the JSON file
with open('openimg\metadata.json', 'r') as file:
    data = json.load(file)

print(type(data))  
print(list(data.keys())) 

<class 'dict'>
['_id', 'name', 'slug', 'version', 'created_at', 'last_loaded_at', 'sample_collection_name', 'persistent', 'media_type', 'group_media_types', 'tags', 'info', 'app_config', 'classes', 'default_classes', 'mask_targets', 'default_mask_targets', 'skeletons', 'sample_fields', 'frame_fields', 'saved_views', 'annotation_runs', 'brain_methods', 'evaluations', 'runs']


In [2]:
import json
import tensorflow as tf

# Load JSON data
with open('openimg\samples.json', 'r') as file:
    data = json.load(file)

samples = data['samples']

samples

[{'_id': {'$oid': '65f7da2d8d1dd746bd684d4e'},
  'filepath': 'data\\68b321aa13d3e402.jpg',
  'tags': ['train'],
  '_media_type': 'image',
  '_rand': 0.9999761021684199,
  'positive_labels': {'_cls': 'Classifications',
   'classifications': [{'_id': {'$oid': '65f7da2d8d1dd746bd684d3b'},
     '_cls': 'Classification',
     'tags': [],
     'label': 'Land vehicle',
     'confidence': 1.0},
    {'_id': {'$oid': '65f7da2d8d1dd746bd684d3d'},
     '_cls': 'Classification',
     'tags': [],
     'label': 'Vehicle',
     'confidence': 1.0},
    {'_id': {'$oid': '65f7da2d8d1dd746bd684d3e'},
     '_cls': 'Classification',
     'tags': [],
     'label': 'Wheel',
     'confidence': 1.0},
    {'_id': {'$oid': '65f7da2d8d1dd746bd684d3f'},
     '_cls': 'Classification',
     'tags': [],
     'label': 'Car',
     'confidence': 1.0}]},
  'negative_labels': {'_cls': 'Classifications',
   'classifications': [{'_id': {'$oid': '65f7da2d8d1dd746bd684d3c'},
     '_cls': 'Classification',
     'tags': [],
    

In [3]:
def get_image_path(relative_filepath):
    base_path = "openimg/"
 
    relative_filepath = relative_filepath.replace("\\", "/")
    return base_path + relative_filepath

In [4]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

data = []

for sample in samples:
    img_path = get_image_path(sample["filepath"])
    img = cv2.imread(img_path)
    
    if img is None:
        continue 

    for detection in sample["detections"]["detections"]:
        label = detection["label"]
        x, y, w, h = detection["bounding_box"]
        x = int(x * img.shape[1])
        y = int(y * img.shape[0])
        w = int(w * img.shape[1])
        h = int(h * img.shape[0])

        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (224, 224)) 
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        roi = preprocess_input(roi)
        
        data.append((roi, label))

In [5]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False)

def extract_features(data):
    features = []
    for item in data:
        roi, label = item
        feature = model.predict(roi)
        features.append((feature, label))
    return features

features = extract_features(data)

1/1 [==============================] - 0s 83ms/step


In [6]:
from sklearn.model_selection import train_test_split
from keras.models import save_model, load_model
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

X = [feature[0].flatten() for feature in features]
y = [label for _, label in features]

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

joblib.dump(clf, 'lr_classifier.joblib')
joblib.dump(le, 'label_encoder.joblib')

clf_loaded = joblib.load('lr_classifier.joblib')
le_loaded = joblib.load('label_encoder.joblib')

In [8]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

y_pred = clf_loaded.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")

Accuracy: 0.532967032967033
Recall: 0.532967032967033
Precision: 0.4518537872384026
F1 Score: 0.4718278045500817


d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
